In [1]:
import pandas as pd
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from supertree import SuperTree

import matplotlib.pyplot as plt
import sys
sys.path.append('..')

from src.utils_step_1 import *

In [2]:
df = pd.read_excel('..\data\dataproject2024.xlsx', index_col=0)

In [3]:
PATH = '..\data\dataproject2024.xlsx'
target = 'PD'

X, y = load_data(PATH, target)

In [ ]:
X.head()

In [ ]:
model = DecisionTreeRegressor(max_depth=7, random_state=0)
model.fit(X, y)

r2_score(y, model.predict(X))

In [ ]:
super_tree = SuperTree(model, X, y)

super_tree.show_tree()
super_tree.save_html('../assets/supertree.html')

# Intepretation
- `Job tenure` appears to be the most important feature when explaining the model's decisions. The threshold is set at 3.5, and a low job tenure is associated with a higher default risk (group avg. of ~30% versus ~13% for the complementary group)
- `Funding amount`, as well as (previous) `Credit event`, also strongly influence the model
- A decision tree of `depth = 7` allows to mimick the black box's behaviour with a .58 r2

# Surrogate model on black-box

In [ ]:
data_blackbox = pd.read_csv("..\Data\catboost_output.csv")
data_blackbox.head(5)

In [8]:
PATH = '..\data\catboost_output'
target = 'pred_default'

X = data_blackbox.drop(["y_hat", "pred_default", "Unnamed: 0"], axis=1)
y = data_blackbox["pred_default"]

In [ ]:
model = DecisionTreeRegressor(max_depth=5, random_state=42)
model.fit(X, y)

r2_score(y, model.predict(X))

# compute mean squared error
mse = ((y - model.predict(X))**2).mean()
print(f"Mean squared error: {mse}")

In [ ]:
super_tree = SuperTree(model, X, y)

super_tree.show_tree()
super_tree.save_html('../assets/supertree_own_model.html')

# Interpretation
Interestingly, while most of the main decision drivers are aligned, being `married` has a higher importance in our black-box model.